In [32]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import LeagueID
from nba_api.stats.library.parameters import SeasonType 
from nba_api.stats.library.parameters import SeasonNullable
current_season = "2024-25"
gamefinder = leaguegamefinder.LeagueGameFinder( 
    game_id_nullable="0022401175",
    season_nullable=SeasonNullable.current_season,
    league_id_nullable=LeagueID.nba,
    season_type_nullable=SeasonType.regular
)
games_df = gamefinder.get_data_frames()[0]

In [36]:
df_filtered = games_df[games_df['GAME_ID'] == "0022401175"]

In [45]:
from nba_api.stats.endpoints import scoreboardv2
game_date_str = '2025-04-10'
scoreboardv2_df = scoreboardv2.ScoreboardV2(game_date=game_date_str, league_id="00")

In [173]:
from nba_api.stats.endpoints import boxscoretraditionalv2
boxscore: pd.DataFrame = boxscoretraditionalv2.BoxScoreTraditionalV2(
                game_id= "0022401172", 
                timeout=0.600
            ).get_data_frames()[0]

boxscore.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS


In [ ]:
class NBAScheduleManager:
	def __init__(self):
		self.headers = {
			'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
		}

	def get_schedule(self, current_season_year=2024):
		"""
		Fetches and processes NBA schedule data for a given season year.
		
		Args:
			current_season_year (int): The year of the season (default: 2024)
			
		Returns:
			pandas.DataFrame: Processed schedule data
		"""
		import requests
		import pandas as pd
		from nba_api.stats.library.parameters import LeagueID

		# Define the URL for the NBA schedule data
		url = f"http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/{current_season_year}/league/{LeagueID.default}_full_schedule.json"
		
		# Get the data
		r = requests.get(url)
		data = r.json()

		# Initialize an empty list to store all games
		all_games_list = []
		
		# Iterate through the league schedule data
		for month in data['lscd']:
			# Extract games from each month
			if 'mscd' in month and 'g' in month['mscd']:
				games = month['mscd']['g']
				all_games_list.extend(games)

		# Convert the list of games to a DataFrame
		games_df = pd.json_normalize(all_games_list)

		# Rename the columns to be more readable
		games_df = games_df.rename(columns={
			'gid': 'game_id',
			'seri': 'playoffs_desc',
			'st': 'game_status',
			'stt': 'game_status_text',
			'gdte': 'game_date',
			'h.tid': 'home_team_id',
			'h.ta': 'home_team_tricode',
			'v.tid': 'visitor_team_id',
			'v.ta': 'visitor_team_tricode',
		})

		# Flag if games are in the playoffs
		games_df['is_playoffs'] = games_df['game_id'].str.startswith('004')
		
		# Select and order columns
		games_df = games_df[[
			"game_id",
			"is_playoffs",
			"playoffs_desc",
			"game_date",
			"home_team_id",
			"home_team_tricode",
			"visitor_team_id",
			"visitor_team_tricode",
			"game_status",
			"game_status_text"
		]]

		return games_df

In [160]:
games_df[games_df['game_id']== "0022401185"]

,game_id,is_playoffs,playoffs_desc,game_date,home_team_id,home_team_tricode,visitor_team_id,visitor_team_tricode,game_status,game_status_text
736,0022401185,False,,2025-04-11,1610612747,LAL,1610612745,HOU,3,Final


In [156]:
schedule_df

,game_id,is_playoffs,playoffs_desc,game_date,home_team_id,home_team_tricode,visitor_team_id,visitor_team_tricode,game_status,game_status_text
0,0022400455,False,,2025-01-01,1610612765,DET,1610612753,ORL,3,Final
1,0022400456,False,,2025-01-01,1610612764,WAS,1610612741,CHI,3,Final
2,0022400457,False,,2025-01-01,1610612748,MIA,1610612740,NOP,3,Final
3,0022400458,False,,2025-01-01,1610612752,NYK,1610612762,UTA,3,Final
4,0022400459,False,,2025-01-01,1610612761,TOR,1610612751,BKN,3,Final
...,...,...,...,...,...,...,...,...,...,...
1307,0022400450,False,,2024-12-31,1610612754,IND,1610612749,MIL,3,Final
1308,0022400451,False,,2024-12-31,1610612759,SAS,1610612746,LAC,3,Final
1309,0022400452,False,,2024-12-31,1610612760,OKC,1610612750,MIN,3,Final
1310,0022400453,False,,2024-12-31,1610612756,PHX,1610612763,MEM,3,Final
